In [ ]:
!pip install focal-loss

In [2]:
import pandas as pd
import numpy as np
import random
import gc
from focal_loss import BinaryFocalLoss
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [3]:
def pT_classes(x):
    if x<=25:
        return 0
    else:
        return 1

df = pd.read_csv('../input/cmsdata/CMS_trigger.csv').drop(columns=['Unnamed: 0'])
df['No. of non-hits'] = df[['Mask_'+str(i) for i in range(12)]].sum(axis = 1)
df = df[df['No. of non-hits']<9].reset_index(drop=True)
df['1/pT'] = df['q/pt'].abs()
df['pT'] = 1/df['1/pT']
df['pT_classes'] = df['pT'].apply(pT_classes)

features = ['Phi_'+str(i) for i in range(12)] + ['Theta_'+str(i) for i in range(12)]
labels_1 = ['1/pT']
labels_2 = ['BendingAngle_'+str(i) for i in [0,1,9,10,11]]
labels_3 = ['pT_classes']
labels_4 = ['PatternStraightness']
masks = ['Mask_'+str(i) for i in range(12)]

scaler_1 = StandardScaler()
df[features] = scaler_1.fit_transform(df[features])
df[features] = df[features].fillna(0)

scaler_2 = MinMaxScaler()
df[labels_2] = scaler_2.fit_transform(df[labels_2])
df[labels_2] = df[labels_2].fillna(0)

scaler_3 = MinMaxScaler()
df[labels_4] = scaler_3.fit_transform(df[labels_4])
df[labels_4] = df[labels_4].fillna(0)

df = df.sample(frac = 1, random_state = 242).reset_index(drop = True)

In [4]:
shuffled_list = list(range(len(df)))
random.Random(242).shuffle(shuffled_list)
shuffled_list = np.array_split(np.array(shuffled_list), 10)

In [5]:
OOF_preds = pd.DataFrame()
OOF_preds['row'] = []
OOF_preds['true_value'] = []
OOF_preds['preds'] = []

In [6]:
def FCNN(X_train, Y1_train, Y2_train, Y3_train, Y4_train):
    p = 0.1
    I = Input(shape=(36))
    x = Dense(1024,activation='relu')(I)
    x = BatchNormalization()(x)
#     x = Dropout(p)(x)
    x = Dense(128,activation='relu')(x)
    x = BatchNormalization()(x)
#     x = Dropout(p)(x)
    x_ = Dense(128,activation='relu')(x)
    x = x + x_ 
    x = BatchNormalization()(x)
#     x = Dropout(p)(x)
    x1 = Dense(128,activation='relu')(x)
    x1 = BatchNormalization()(x1)
    x2 = Dense(128,activation='relu')(x)
    x2 = BatchNormalization()(x2)
    x3 = Dense(128,activation='relu')(x)
    x3 = BatchNormalization()(x3)
    x4 = Dense(128,activation='relu')(x)
    x4 = BatchNormalization()(x4)
    O1 = Dense(1,activation='sigmoid')(x1)
    O2 = Dense(5,activation='sigmoid')(x2)
    O3 = Dense(1,activation='sigmoid')(x3)
    O4 = Dense(1,activation='sigmoid')(x4)
    model = Model(inputs=I, outputs=[O1,O2,O3,O4])

    batch_size=128
    path = "model.h5"

    checkpoint = ModelCheckpoint(path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    early_stop = EarlyStopping(monitor='val_loss',patience=10,verbose=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=2,verbose=True)

    model.compile(optimizer = 'adam', loss=['mse','mse',BinaryFocalLoss(gamma=4),'mse'])
    model.summary()

    history = model.fit(x=X_train, y=[Y1_train,Y2_train,Y3_train,Y4_train], batch_size=batch_size, epochs=5, verbose=0, validation_split=0.2, callbacks=[checkpoint,early_stop,reduce_lr])
    
    k = sorted(list(history.history.keys()))
    plt.plot(history.history[k[4]])
    plt.plot(history.history[k[10]])
    plt.legend(['Total loss','Val loss'])
    plt.show()
    plt.plot(history.history[k[0]])
    plt.plot(history.history[k[6]])
    plt.legend(['Momentum MSE','Val Momentum MSE'])
    plt.show()
    plt.plot(history.history[k[1]])
    plt.plot(history.history[k[7]])
    plt.legend(['Bending Angle MSE','Val Bending Angle MSE'])
    plt.show()
    plt.plot(history.history[k[2]])
    plt.plot(history.history[k[8]])
    plt.legend(['Focal loss','Val Focal loss'])
    plt.show()
    plt.plot(history.history[k[3]])
    plt.plot(history.history[k[9]])
    plt.legend(['Pattern Straightness MSE','Val Pattern Straightness MSE'])
    plt.show()
    
    return model

In [1]:
for i in range(0,10):
    X_train = df[features+masks].iloc[np.concatenate([shuffled_list[j] for j in range(10) if j not in (i,100)])]
    Y1_train = df[labels_1].iloc[np.concatenate([shuffled_list[j] for j in range(10) if j not in (i,100)])]
    Y2_train = df[labels_2].iloc[np.concatenate([shuffled_list[j] for j in range(10) if j not in (i,100)])]
    Y3_train = df[labels_3].astype('float32').iloc[np.concatenate([shuffled_list[j] for j in range(10) if j not in (i,100)])]
    Y4_train = df[labels_4].iloc[np.concatenate([shuffled_list[j] for j in range(10) if j not in (i,100)])]

    X_test = df[features+masks].iloc[shuffled_list[i]]
    Y1_test = df[labels_1].iloc[shuffled_list[i]]
    Y2_test = df[labels_2].iloc[shuffled_list[i]]
    Y3_test = df[labels_3].astype('float32').iloc[shuffled_list[i]]
    Y4_test = df[labels_4].iloc[shuffled_list[i]]
    
    model = FCNN(X_train, Y1_train, Y2_train, Y3_train, Y4_train)
    
    P = model.predict(X_test)
    
    test_preds = P[0].reshape((len(X_test)))
    
    OOF_preds_ = pd.DataFrame()
    OOF_preds_['row'] = shuffled_list[i]
    OOF_preds_['true_value'] = Y1_test['1/pT'].values
    OOF_preds_['preds'] = test_preds
    OOF_preds = pd.concat([OOF_preds,OOF_preds_],axis = 0).reset_index(drop = True)
    
    print(classification_report(Y3_test.to_numpy().reshape((len(Y3_test))),np.array(P[2].reshape((len(Y3_test)))>0.5)*1))
    print(confusion_matrix(Y3_test.to_numpy().reshape((len(Y3_test))),np.array(P[2].reshape((len(Y3_test)))>0.5)*1))
    
    gc.collect()

In [ ]:
OOF_preds = OOF_preds.sort_values(by = 'row').reset_index(drop = True)
OOF_preds.to_csv('OOF_preds.csv')